In [7]:
# Sample dataset with 30 facts about Berlin
documents = [
    "Berlin is the capital and largest city of Germany by both area and population.",
    "Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.",
    "The Berlin Wall, which divided the city from 1961 to 1989, was a significant Cold War symbol.",
    "Berlin has more bridges than Venice, with around 1,700 bridges.",
    "The city's Zoological Garden is the most visited zoo in Europe and one of the most popular worldwide.",
    "Berlin's Museum Island is a UNESCO World Heritage site with five world-renowned museums.",
    "The Reichstag building houses the German Bundestag (Federal Parliament).",
    "Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.",
    "The Berlin Marathon is one of the world's largest and most popular marathons.",
    "Berlin's public transportation system includes buses, trams, U-Bahn (subway), and S-Bahn (commuter train).",
    "The Brandenburg Gate is an iconic neoclassical monument in Berlin.",
    "Berlin has a thriving startup ecosystem and is considered a major tech hub in Europe.",
    "The city hosts the Berlinale, one of the most prestigious international film festivals.",
    "Berlin has more than 180 kilometers of navigable waterways.",
    "The East Side Gallery is an open-air gallery on a remaining section of the Berlin Wall.",
    "Berlin's Tempelhofer Feld, a former airport, is now a public park and recreational area.",
    "The TV Tower at Alexanderplatz offers panoramic views of the city.",
    "Berlin's Tiergarten is one of the largest urban parks in Germany.",
    "Checkpoint Charlie was a famous crossing point between East and West Berlin during the Cold War.",
    "Berlin is home to numerous theaters, including the Berliner Ensemble and the Volksbühne.",
    "The Berlin Philharmonic Orchestra is one of the most famous orchestras in the world.",
    "Berlin has a vibrant nightlife scene, with countless bars, clubs, and music venues.",
    "The Berlin Cathedral is a major Protestant church and a landmark of the city.",
    "Charlottenburg Palace is the largest palace in Berlin and a major tourist attraction.",
    "Berlin's Alexanderplatz is a large public square and transport hub in central Berlin.",
    "Berlin is known for its street art, with many murals and graffiti artworks around the city.",
    "The Gendarmenmarkt is a historic square in Berlin featuring the Konzerthaus, French Cathedral, and German Cathedral.",
    "Berlin has a strong coffee culture, with numerous cafés throughout the city.",
    "The Berlin TV Tower is the tallest structure in Germany, standing at 368 meters.",
    "Berlin's KaDeWe is one of the largest and most famous department stores in Europe.",
    "The Berlin U-Bahn network has 10 lines and serves 173 stations.",
    "Berlin has a population of over 3.6 million people.",
    "The city of Berlin covers an area of 891.8 square kilometers.",
    "Berlin has a temperate seasonal climate.",
    "The Berlin International Film Festival, also known as the Berlinale, is one of the world's leading film festivals.",
    "Berlin is home to the Humboldt University, founded in 1810.",
    "The Berlin Hauptbahnhof is the largest train station in Europe.",
    "Berlin's Tegel Airport closed in 2020, and operations moved to Berlin Brandenburg Airport.",
    "The Spree River runs through the center of Berlin.",
    "Berlin is twinned with Los Angeles, California, USA.",
    "The Berlin Botanical Garden is one of the largest and most important botanical gardens in the world.",
    "Berlin has over 2,500 public parks and gardens.",
    "The Victory Column (Siegessäule) is a famous monument in Berlin.",
    "Berlin's Olympic Stadium was built for the 1936 Summer Olympics.",
    "The Berlin State Library is one of the largest libraries in Europe.",
    "The Berlin Dungeon is a popular tourist attraction that offers a spooky look at the city's history.",
    "Berlin's economy is based on high-tech industries and the service sector.",
    "Berlin is a major center for culture, politics, media, and science.",
    "The Berlin Wall Memorial commemorates the division of Berlin and the victims of the Wall.",
    "The city has a large Turkish community, with many residents of Turkish descent.",
    "Berlin's Mauerpark is a popular park known for its flea market and outdoor karaoke sessions.",
    "The Berlin Zoological Garden is the oldest zoo in Germany, opened in 1844.",
    "Berlin is known for its diverse culinary scene, including many vegan and vegetarian restaurants.",
    "The Berliner Dom is a baroque-style cathedral located on Museum Island.",
    "The DDR Museum in Berlin offers interactive exhibits about life in East Germany.",
    "Berlin has a strong cycling culture, with many dedicated bike lanes and bike-sharing programs.",
    "Berlin's Tempodrom is a multi-purpose event venue known for its unique architecture.",
    "The Berlinische Galerie is a museum of modern art, photography, and architecture.",
    "Berlin's Volkspark Friedrichshain is the oldest public park in the city, established in 1848.",
    "The Hackesche Höfe is a complex of interconnected courtyards in Berlin's Mitte district, known for its vibrant nightlife and art scene.",
    "Berlin's International Congress Center (ICC) is one of the largest conference centers in the world."
]

Tokenization and Embeddings

In [8]:
!pip install faiss-cpu

In [9]:
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [10]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

In [11]:
def embed_text(text,tokenizer,model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state
        embeddings = embeddings.mean(dim=1)
    return embeddings

In [12]:
document_embeddings = []
for document in documents:
    document_embedding = embed_text(document,tokenizer,model)
    document_embeddings.append(document_embedding)

In [13]:
document_embeddings = torch.cat(document_embeddings, dim=0).cpu().numpy()

In [14]:
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

Learn this in details..

In [15]:
def retrieve_documents(query, index, documents, k=5):
    query_embedding = embed_text(query,tokenizer,model)
    distances, indices = index.search(query_embedding, k)
    return [documents[i] for i in indices[0]], distances[0]

In [18]:
query = "What is the capital of Germany?"
retrieved_documents, distances = retrieve_documents(query, index, documents)
print("Retrieved documents:", retrieved_documents)
print("Distances:", distances)

Retrieved documents: ['Berlin is the capital and largest city of Germany by both area and population.', 'Berlin is a major center for culture, politics, media, and science.', 'Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.', "Berlin's Tiergarten is one of the largest urban parks in Germany.", 'Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.']
Distances: [36.25392  45.818718 48.381382 51.32155  51.4573  ]


In [19]:
from transformers import AutoModelForCausalLM

In [20]:
gen_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gen_model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [21]:
gen_tokenizer.pad_token = gen_tokenizer.eos_token
#

In [22]:
context = "".join(retrieved_documents)
context

"Berlin is the capital and largest city of Germany by both area and population.Berlin is a major center for culture, politics, media, and science.Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.Berlin's Tiergarten is one of the largest urban parks in Germany.Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures."

Check this one out

In [28]:
def generate_answer(query, context, tokenizer, model):
  input_text = f"Context: {context}\nQuestion: {query}\nAnswer:"
  input = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
  input_ids = input["input_ids"]
  attention_mask = (input_ids != tokenizer.pad_token_id).long()
  outputs = gen_model.generate(input_ids, attention_mask=attention_mask, max_length=100, pad_token_id=tokenizer.eos_token_id)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [29]:
generated_answer = generate_answer(query, context, gen_tokenizer, gen_model)
generated_answer

"Context: Berlin is the capital and largest city of Germany by both area and population.Berlin is a major center for culture, politics, media, and science.Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.Berlin's Tiergarten is one of the largest urban parks in Germany.Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.\nQuestion: What is the capital of Germany?\nAnswer: Berlin"